%pip install sqlalchemy
%pip install pymysql
%pip install cryptography
%pip install mysql-connector-python



# Exhaustive Performance

In [14]:
import mysql.connector
mydb = mysql.connector.connect(
  host="ubuntu.sesar.int",
  user="root",
  password="r00t",
  port="3309",
    database="pippo"
)
mycursor = mydb.cursor()

TABLE = "pippo.mytable_gamma_vbad"
MAX_NUMBER_OF_NODES = 7
MAX_NUMBER_OF_SERVICES = 6

In [15]:
result_dict = {}

mycursor.execute(f"""SELECT number_of_services, number_of_nodes, avg(execution_time) as execution
        FROM {TABLE}
        WHERE window_size = number_of_nodes
        GROUP BY number_of_nodes, number_of_services
        ORDER BY number_of_services
    """)

for row in mycursor:
    number_of_services = row[0]
    number_of_nodes = row[1]
    execution_time = row[2]

    if number_of_services not in result_dict:
        result_dict[number_of_services] = [execution_time]
    else:
        result_dict[number_of_services].append(execution_time)

# Print the formatted output
with open(f"exhaustive_performance_{TABLE}.dat", "w") as file:
    for number_of_nodes, execution_times in result_dict.items():
        starred = " ".join(map(str, execution_times))
        file.write(f"{number_of_nodes} {starred}\n")
        print(number_of_nodes, starred)



2.0 5692.765424333333 395.1175735555555 710.7383522222221 1410.558955111111 2702.4940743333336
3.0 1154.4522102222222 3483.5072357777776 10271.13870577778 30933.83246977777 92190.83902144444
4.0 10197.356466333333 42897.56829722222 175430.73173066665 706069.2885016666 2551.260210111111
5.0 130421.33068500001 656147.0523581111 3359479.534072889 5154.821100000001 26182.719725333332
6.0 12261119.104276335 9037.278735 54802.827300666664 320049.89177344437 1978217.6977802226
7.0 14352.719471 98833.08043811111 702823.2813468889 5107027.250075889


# Window Time Performance 

In [16]:


mycursor = mydb.cursor()
#result_dict = {}
for n in range(2,8):

    mycursor.execute(f"""SELECT window_size,number_of_services, number_of_nodes, avg(execution_time) as execution
            FROM  {TABLE}
            WHERE number_of_nodes =  {n}
            GROUP BY number_of_nodes, number_of_services,window_size
            ORDER BY window_size

        """)


    result_dict = {}
    for row in mycursor:
        window_size, number_of_services, number_of_nodes, execution_time  = row
        if window_size not in result_dict:
            result_dict[window_size] = {number_of_services: [execution_time]}
        else:
            if number_of_services not in result_dict[window_size]:
                result_dict[window_size][number_of_services] = [execution_time]
            else:
                result_dict[window_size][number_of_services].append(execution_time)

    with open(f"window_time_performance_{TABLE}_n{n}.dat", "w") as file:
        for window_size, services in result_dict.items():
            for number_of_services, execution_times in services.items():
                starred = " ".join(map(str, execution_times))
                file.write(f"{number_of_services} {starred}\n")
                print(window_size, number_of_services, starred)

            file.write("\n\n")


    # Print the formatted output
    # with open(f"window_time_performance_{TABLE}_n{n}_w{w}.dat", "w") as file:
    #     for number_of_nodes, execution_times in result_dict.items():
    #         starred = " ".join(map(str, execution_times))
    #         file.write(f"{number_of_nodes} {starred}\n")
    #         print(number_of_nodes, starred)




1.0 5.0 692.7665155555557
1.0 7.0 946.4128094444445
1.0 2.0 450.07476266666663
1.0 4.0 563.4258538888888
1.0 6.0 813.2318067777777
1.0 3.0 460.07102444444445
2.0 3.0 787.2276863333333
2.0 5.0 2025.007479111111
2.0 7.0 4016.3408646666667
2.0 2.0 487.625558
2.0 4.0 1296.9883637777775
2.0 6.0 2942.9121785555562
3.0 6.0 9037.278735
3.0 3.0 1154.4522102222222
3.0 5.0 5154.821100000001
3.0 7.0 14352.719471
3.0 2.0 395.1175735555555
3.0 4.0 2551.260210111111
1.0 3.0 615.932865
1.0 7.0 1214.6095004444446
1.0 2.0 473.61822422222224
1.0 6.0 1074.8752433333334
1.0 5.0 920.3477274444444
1.0 4.0 736.3195102222222
2.0 4.0 1902.3452684444446
2.0 3.0 1171.1842572222222
2.0 7.0 5766.9849393333325
2.0 2.0 588.1730364444443
2.0 6.0 4317.91663411111
2.0 5.0 2968.123852444444
3.0 5.0 10202.889924777779
3.0 4.0 5007.172827333333
3.0 3.0 2242.830816333333
3.0 7.0 27678.333762333332
3.0 2.0 719.1114527777777
3.0 6.0 17616.595100333332
4.0 5.0 26182.719725333332
4.0 4.0 10197.356466333333
4.0 3.0 3483.50723577

# Window Quality Performance

In [17]:
mycursor = mydb.cursor()
#result_dict = {}
for n in range(2,MAX_NUMBER_OF_NODES+1):
    for w in range(1,n+1):
        query = f"""SELECT number_of_services, number_of_nodes, avg(metric)
                FROM {TABLE}
                WHERE number_of_nodes = {n} AND window_size = {w}
                GROUP BY number_of_nodes, number_of_services,window_size
                ORDER BY number_of_services
            """
        mycursor.execute(query)

        result_dict = {}
        for row in mycursor:
            number_of_services = row[0]
            number_of_nodes = row[1]
            execution_time = row[2]

            if number_of_services not in result_dict:
                result_dict[number_of_services] = [execution_time]
            else:
                result_dict[number_of_services].append(execution_time)

        # Print the formatted output
        with open(f"window_quality_performance_{TABLE}_n{n}_w{w}.dat", "w") as file:
            for number_of_nodes, execution_times in result_dict.items():
                starred = " ".join(map(str, execution_times))
                file.write(f"{number_of_nodes} {starred}\n")
                print(number_of_nodes, starred)

2.0 0.1167143853933226
3.0 0.0823122266149606
4.0 0.08759954986206392
5.0 0.07259457461157025
6.0 0.06826242594438976
7.0 0.06743699336074414
2.0 0.1063197881903679
3.0 0.0823122266149606
4.0 0.08389520001812129
5.0 0.06890374964579507
6.0 0.06431963580364027
7.0 0.06837047086895659
2.0 0.10555768841513095
3.0 0.0823122266149606
4.0 0.08389520001812129
5.0 0.0677913942937069
6.0 0.06431963580364027
7.0 0.06392027280859111
2.0 0.1626541983998534
3.0 0.11620592732651391
4.0 0.11747322236785035
5.0 0.09168404464707584
6.0 0.08909558432077683
7.0 0.09168813754887264
2.0 0.14953231213941937
3.0 0.10062356761849463
4.0 0.10522373343195601
5.0 0.09120379473607837
6.0 0.08182890216757396
7.0 0.08052530590286965
2.0 0.14391982609231174
3.0 0.10228580286760171
4.0 0.10263758084744323
5.0 0.0846309808235347
6.0 0.08005150492662955
7.0 0.08131139434684108
2.0 0.1436971623789497
3.0 0.10062356761849463
4.0 0.10240312502493469
5.0 0.0846309808235347
6.0 0.07796212658841932
7.0 0.07931297086151891
2.

In [18]:


mycursor = mydb.cursor()
#result_dict = {}
for n in range(2,6):
    for w in range(1,n):
        query = f"""SELECT number_of_services, number_of_nodes, avg(percentage)
                FROM pippo.mytable
                WHERE number_of_nodes = {n} AND window_size = {w}
                GROUP BY number_of_nodes, number_of_services,window_size
                ORDER BY number_of_services
            """
        mycursor.execute(query)

        result_dict = {}
        for row in mycursor:
            number_of_services = row[0]
            number_of_nodes = row[1]
            execution_time = row[2]

            if number_of_services not in result_dict:
                result_dict[number_of_services] = [execution_time]
            else:
                result_dict[number_of_services].append(execution_time)

        # Print the formatted output
        with open(f"window_percentage_performance_n{n}_w{w}.dat", "w") as file:
            for number_of_nodes, execution_times in result_dict.items():
                starred = " ".join(map(str, execution_times))
                file.write(f"{wino} {number_of_nodes} {starred}\n")
                print(number_of_nodes, starred)


ProgrammingError: 1146 (42S02): Table 'pippo.mytable' doesn't exist

In [ ]:
mydb.close()

In [ ]:


mycursor = mydb.cursor()
#result_dict = {}
for n in range(2,6):
    for w in range(1,n):
        query = f"""SELECT number_of_services, number_of_nodes, avg(percentage)
                FROM pippo.mytable
                WHERE number_of_nodes = {n} AND window_size = {w}
                GROUP BY number_of_nodes, number_of_services,window_size
                ORDER BY number_of_services
            """
        mycursor.execute(query)

        result_dict = {}
        for row in mycursor:
            number_of_services = row[0]
            number_of_nodes = row[1]
            execution_time = row[2]

            if number_of_services not in result_dict:
                result_dict[number_of_services] = [execution_time]
            else:
                result_dict[number_of_services].append(execution_time)

        # Print the formatted output
        with open(f"window_percentage_performance_n{n}_w{w}.dat", "w") as file:
            for number_of_nodes, execution_times in result_dict.items():
                starred = " ".join(map(str, execution_times))
                file.write(f"{number_of_nodes} {starred}\n")
                print(number_of_nodes, starred)


mydb.close()





OperationalError: MySQL Connection not available.